In [1]:
import backtrader as bt
import yfinance as yf
import pandas as pd
import datetime

In [2]:
# 下載股票資料
df = yf.download('AAPL', start='2020-01-01', end='2023-01-01')

# 清理掉多重索引
df.columns = df.columns.droplevel(1)
df

/var/folders/k0/mjnvdkdj7sn_zbtz76h68xr40000gn/T/ipykernel_54153/534702881.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download('AAPL', start='2020-01-01', end='2023-01-01')
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,72.468269,72.528589,71.223267,71.476607,135480400
2020-01-03,71.763718,72.523746,71.539330,71.696160,146322800
2020-01-06,72.335564,72.374169,70.634547,70.885479,118387200
2020-01-07,71.995346,72.600952,71.775781,72.345197,108872000
2020-01-08,73.153519,73.455118,71.698604,71.698604,132079200
...,...,...,...,...,...
2022-12-23,129.900299,130.451974,127.713291,128.974267,63814900
2022-12-27,128.097458,129.456953,126.806930,129.427400,69007800
2022-12-28,124.166794,129.082630,123.999322,127.742842,85438400


In [3]:
# 改成 bt 可接受的格式
data = bt.feeds.PandasData(dataname=df)

In [4]:
# 設定回測執行的大腦
cerebro = bt.Cerebro()

# 將資料加到回測大腦中
cerebro.adddata(data)

# 設定初始資金
cerebro.broker.setcash(100000)

# 設定交易手續費
cerebro.broker.setcommission(commission=0.001) 

In [5]:
# 建立簡易的交易策略
import backtrader as bt

class MyFirstStrategy(bt.Strategy):

    params = (
        ('exitbars', 5),
    )

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} - {txt}')

    def __init__(self):
        # close price reference
        self.dataclose = self.datas[0].close

        # track pending order
        self.order = None

        self.log('MyFirstStrategy Initialized')

    def notify_order(self, order):
        # order submitted/accepted
        if order.status in [order.Submitted, order.Accepted]:
            self.log(f'ORDER {order.getstatusname()}')
            return

        # order executed
        if order.status == order.Completed:
            if order.isbuy():
                self.log(
                    f'BUY EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Value: {order.executed.value:.2f}, '
                    f'Comm: {order.executed.comm:.2f}'
                )
            else:
                self.log(
                    f'SELL EXECUTED, Price: {order.executed.price:.2f}, '
                    f'Value: {order.executed.value:.2f}, '
                    f'Comm: {order.executed.comm:.2f}'
                )

            # clear pending order
            self.order = None

        # canceled, rejected, margin
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log(f'Order {order.getstatusname()}')
            self.order = None

    def next(self):
        if len(self) % 100 == 0:
            self.log(f'處理第 {len(self)} 根 K 棒')
        # 若有 pending order，不做任何事
        if self.order:
            return

        # 如果是第一根 K 且目前沒有持倉 → BUY
        if len(self) == 1:
            if not self.position:
                self.log(f'BUY CREATE, {self.dataclose[0]:.2f}')
                self.order = self.buy()
                return

        # 持有 exitbars 根 K 後 → SELL
        if len(self) >= self.params.exitbars + 1:
            if self.position:
                self.log(f'SELL CREATE, {self.dataclose[0]:.2f}')
                self.order = self.sell()


In [6]:
# 執行策略
print("\nRunning backtest...")

print(f"Starting Portfolio Value: {cerebro.broker.getvalue():,.2f}")

# 加入策略
cerebro.addstrategy(MyFirstStrategy)
# Run the backtest
results = cerebro.run()

# Get the strategy instance
first_strategy = results[0]

print(f"Final Portfolio Value: {cerebro.broker.getvalue():,.2f}")
print("Backtest complete.")


Running backtest...
Starting Portfolio Value: 100,000.00
2022-12-30 - MyFirstStrategy Initialized
2020-01-02 - BUY CREATE, 72.47
2020-01-03 - ORDER Submitted
2020-01-03 - ORDER Accepted
2020-01-03 - BUY EXECUTED, Price: 71.70, Value: 71.70, Comm: 0.07
2020-01-09 - SELL CREATE, 74.71
2020-01-10 - ORDER Submitted
2020-01-10 - ORDER Accepted
2020-01-10 - SELL EXECUTED, Price: 74.94, Value: 71.70, Comm: 0.07
2020-05-26 - 處理第 100 根 K 棒
2020-10-15 - 處理第 200 根 K 棒
2021-03-11 - 處理第 300 根 K 棒
2021-08-03 - 處理第 400 根 K 棒
2021-12-23 - 處理第 500 根 K 棒
2022-05-18 - 處理第 600 根 K 棒
2022-10-11 - 處理第 700 根 K 棒
Final Portfolio Value: 100,003.10
Backtest complete.


In [10]:
# 透過 matplotlib 繪製圖表
import matplotlib.pyplot as plt

print("\nGenerating plot...")

try:
    cerebro.plot(
        style='candlestick',
        barup='red',
        bardown='green',
        volume=True,
        iplot=False,   # 背後使用 matplotlib，而非 Bokeh
        show=False
    )
    print("Plot displayed.")

except Exception as e:
    print(f"Could not plot results: {e}")
    print("Make sure matplotlib is installed and working correctly.")


Generating plot...
Plot displayed.
